In [1]:
import os
import sys

def set_root_path():
    if os.getcwd().endswith('figures'): 
        os.chdir('../')
set_root_path()
sys.path.append('python/')
sys.path = [p for p in sys.path if "unbiased-inverse-volume-rendering" not in p]
print(os.getcwd())
print(sys.executable)
os.environ.pop("PYTHONPATH")

c:\Users\mayuf\source\repos\practical-inverse-rendering-of-textured-and-translucent-appearance
c:\Users\mayuf\anaconda3\envs\practical-inverse-rendering\python.exe


'C:\\Users\\mayuf\\source\\repos\\unbiased-inverse-volume-rendering\\mitsuba3\\build\\Release\\python'

In [2]:
from pathlib import Path
import mitsuba as mi
import numpy as np
mi.set_variant('cuda_ad_rgb')

from practical_reconstruction import optimization_cli
from core import integrators
from core import bsdfs
from core import textures

integrators.register()
bsdfs.register()
textures.register()

In [ ]:
def _update_light_position(params, camera_idx):
	light_positions = [
		["7.500000 0.000000 0.000000 0.300000 0.000000 7.471460 0.043578 2.949549 0.000000 -0.653668 0.498097 -35.366814 0.000000 0.000000 0.000000 1.000000"],
		["-7.500000 0.000001 -0.000000 0.300000 0.000001 7.471460 -0.043578 9.050451 -0.000000 -0.653669 -0.498097 34.366814 0.000000 0.000000 0.000000 1.000000"],
		["7.328826 0.166872 0.105630 -7.094067 -0.000000 7.458747 -0.052370 9.665897 -1.593206 0.767620 0.485901 -34.513069 0.000000 0.000000 0.000000 1.000000"],
		["-7.328826 0.166873 -0.105630 7.694067 0.000001 7.458748 0.052370 2.334103 1.593207 0.767619 -0.485901 33.513069 0.000000 0.000000 0.000000 1.000000"],
		["7.494684 0.133494 -0.016586 1.461014 0.000000 6.608728 0.236406 -10.548397 0.282341 -3.543571 0.440270 -31.318865 0.000000 0.000000 0.000000 1.000000"],
		["-7.494684 0.133495 0.016586 -0.861014 0.000001 6.608727 -0.236406 22.548397 -0.282341 -3.543573 -0.440269 30.318865 0.000000 0.000000 0.000000 1.000000"],
		["7.117022 -0.706185 -0.150544 10.838095 -0.000000 7.158140 -0.149236 16.446486 2.366009 2.124227 0.452841 -32.198891 0.000000 0.000000 0.000000 1.000000"],
		["-7.117022 -0.706185 0.150544 -10.238094 -0.000000 7.158140 0.149235 -4.446486 -2.366009 2.124224 -0.452841 31.198891 0.000000 0.000000 0.000000 1.000000"],
		["6.304254 -0.738532 0.266341 -18.343878 0.000000 7.375046 0.090889 -0.362262 -4.062806 -1.145979 0.413281 -29.429699 0.000000 0.000000 0.000000 1.000000"],
		["-6.304254 -0.738533 -0.266341 18.943876 -0.000000 7.375046 -0.090890 12.362262 4.062806 -1.145981 -0.413281 28.429699 0.000000 0.000000 0.000000 1.000000"],
		["6.311899 1.645186 -0.246786 17.575043 0.000000 6.853625 0.203063 -8.214446 4.050918 -2.563432 0.384528 -27.416950 0.000000 0.000000 0.000000 1.000000"],
		["-6.311899 1.645188 0.246786 -16.975044 -0.000000 6.853623 -0.203064 20.214447 -4.050917 -2.563436 -0.384528 26.416950 0.000000 0.000000 0.000000 1.000000"],
		["7.316449 0.928876 0.090842 -6.058943 -0.000000 6.197108 -0.281629 25.714031 -1.649113 4.121049 0.403030 -28.712067 0.000000 0.000000 0.000000 1.000000"],
		["-7.316449 0.928876 -0.090842 6.658943 0.000000 6.197108 0.281629 -13.714031 1.649113 4.121048 -0.403030 27.712067 0.000000 0.000000 0.000000 1.000000"],
		["6.835326 -2.120719 0.149531 -10.167147 0.000000 5.449726 0.343514 -18.046011 -3.086798 -4.696066 0.331117 -23.678185 0.000000 0.000000 0.000000 1.000000"],
		["-6.835326 -2.120720 -0.149531 10.767147 -0.000001 5.449725 -0.343514 30.046011 3.086798 -4.696066 -0.331117 22.678185 0.000000 0.000000 0.000000 1.000000"],
		["5.221693 -0.906111 -0.353791 25.065395 -0.000000 7.393009 -0.084154 11.890759 5.383672 0.878849 0.343147 -24.520285 0.000000 0.000000 0.000000 1.000000"],
		["-5.221694 -0.906112 0.353791 -24.465397 -0.000001 7.393009 0.084154 0.109242 -5.383671 0.878849 -0.343147 23.520285 0.000000 0.000000 0.000000 1.000000"],
		["4.966681 1.186354 0.366210 -25.334677 0.000000 7.330979 -0.105551 13.388588 -5.619793 1.048480 0.323650 -23.155512 0.000000 0.000000 0.000000 1.000000"],
		["-4.966681 1.186353 -0.366210 25.934675 0.000001 7.330980 0.105551 -1.388588 5.619793 1.048478 -0.323650 22.155512 0.000000 0.000000 0.000000 1.000000"],
		["6.495440 2.950367 -0.154267 11.098653 -0.000000 4.628530 0.393428 -21.539946 3.749567 -5.110973 0.267239 -19.206701 0.000000 0.000000 0.000000 1.000000"],
		["-6.495441 2.950367 0.154266 -10.498652 0.000001 4.628530 -0.393428 33.539948 -3.749566 -5.110973 -0.267239 18.206701 0.000000 0.000000 0.000000 1.000000"],
		["6.831079 -2.292678 -0.138724 10.010694 0.000000 5.040548 -0.370242 31.916969 3.096185 5.058310 0.306066 -21.924593 0.000000 0.000000 0.000000 1.000000"],
		["-6.831079 -2.292679 0.138724 -9.410693 -0.000000 5.040547 0.370242 -19.916969 -3.096186 5.058311 -0.306066 20.924593 0.000000 0.000000 0.000000 1.000000"],
		["4.241152 -3.143757 0.355149 -24.560425 -0.000000 6.459154 0.254116 -11.788103 -6.185679 -2.155488 0.243504 -17.545313 0.000000 0.000000 0.000000 1.000000"],
		["-4.241152 -3.143759 -0.355149 25.160423 -0.000000 6.459153 -0.254116 23.788103 6.185679 -2.155488 -0.243504 16.545313 0.000000 0.000000 0.000000 1.000000"],
		["3.585440 1.807922 -0.422301 29.861053 0.000000 7.212013 0.137224 -3.605716 6.587459 -0.984020 0.229851 -16.589569 0.000000 0.000000 0.000000 1.000000"],
		["-3.585441 1.807922 0.422301 -29.261055 -0.000001 7.212012 -0.137225 15.605716 -6.587459 -0.984022 -0.229851 15.589569 0.000000 0.000000 0.000000 1.000000"],
		["4.772033 3.920635 0.283677 -19.557381 -0.000000 5.515674 -0.338804 29.716280 -5.785993 3.233568 0.233964 -16.877499 0.000000 0.000000 0.000000 1.000000"],
		["-4.772034 3.920633 -0.283677 20.157379 0.000000 5.515676 0.338804 -17.716280 5.785991 3.233568 -0.233964 15.877499 0.000000 0.000000 0.000000 1.000000"],
		["7.172763 -2.072851 0.047364 -3.015504 -0.000000 2.431738 0.472989 -27.109219 -2.191226 -6.785274 0.155042 -11.352975 0.000000 0.000000 0.000000 1.000000"],
		["-7.172763 -2.072851 -0.047364 3.615504 0.000001 2.431739 -0.472989 39.109219 2.191226 -6.785274 -0.155043 10.352975 0.000000 0.000000 0.000000 1.000000"],
		["3.266159 -3.596402 -0.380923 26.964638 -0.000000 6.347352 -0.266343 24.643972 6.751460 1.739834 0.184280 -13.399581 0.000000 0.000000 0.000000 1.000000"],
		["-3.266160 -3.596401 0.380923 -26.364639 -0.000001 6.347353 0.266342 -12.643972 -6.751459 1.739834 -0.184280 12.399581 0.000000 0.000000 0.000000 1.000000"],
		["2.297698 -0.342003 0.475411 -32.978809 -0.000001 7.491390 0.023952 4.323370 -7.139369 -0.110069 0.153004 -11.210276 0.000000 0.000000 0.000000 1.000000"],
		["-2.297697 -0.342002 -0.475412 33.578808 -0.000001 7.491390 -0.023952 7.676630 7.139369 -0.110067 -0.153004 10.210276 0.000000 0.000000 0.000000 1.000000"],
		["2.714157 5.183684 -0.312785 22.194952 0.000001 5.032897 0.370705 -19.949331 6.991663 -2.012302 0.121423 -8.999601 0.000000 0.000000 0.000000 1.000000"],
		["-2.714157 5.183686 0.312785 -21.594954 -0.000001 5.032896 -0.370705 31.949331 -6.991663 -2.012303 -0.121423 7.999601 0.000000 0.000000 0.000000 1.000000"],
		["7.122634 2.323870 0.022872 -1.301074 0.000000 1.095397 -0.494638 40.624687 -2.349059 7.046257 0.069352 -5.354651 0.000000 0.000000 0.000000 1.000000"],
		["-7.122635 2.323869 -0.022872 1.901073 -0.000000 1.095398 0.494638 -28.624687 2.349058 7.046257 -0.069352 4.354651 0.000000 0.000000 0.000000 1.000000"],
		["2.099781 -5.825052 0.282132 -19.449261 0.000000 4.408278 0.404514 -22.315975 -7.200064 -1.698781 0.082279 -6.259549 0.000000 0.000000 0.000000 1.000000"],
		["-2.099783 -5.825052 -0.282132 20.049259 0.000000 4.408278 -0.404514 34.315975 7.200063 -1.698783 -0.082279 5.259549 0.000000 0.000000 0.000000 1.000000"],
		["1.080882 -0.334599 -0.494277 34.899403 -0.000000 7.492374 -0.022542 7.577931 7.421704 0.048731 0.071986 -5.538995 0.000000 0.000000 0.000000 1.000000"],
		["-1.080884 -0.334599 0.494277 -34.299404 -0.000000 7.492374 0.022542 4.422069 -7.421704 0.048730 -0.071986 4.538995 0.000000 0.000000 0.000000 1.000000"],
		["0.929983 3.678388 0.431301 -29.891064 -0.000001 6.519829 -0.247133 23.299314 -7.442119 0.459659 0.053896 -4.272741 0.000000 0.000000 0.000000 1.000000"],
		["-0.929983 3.678386 -0.431301 30.491062 0.000000 6.519831 0.247133 -11.299314 7.442119 0.459659 -0.053896 3.272741 0.000000 0.000000 0.000000 1.000000"],
		["-0.882609 7.263825 -0.109704 7.979250 0.000000 1.657068 0.487643 -28.135040 7.447886 0.860797 -0.013000 0.410026 0.000000 0.000000 0.000000 1.000000"],
		["0.882609 7.263825 0.109704 -7.379250 0.000000 1.657068 -0.487643 40.135040 -7.447886 0.860797 0.013000 -1.410026 0.000000 0.000000 0.000000 1.000000"],
		["0.307662 -6.150860 -0.285363 20.275440 -0.000000 4.284057 -0.410403 34.728199 7.493687 0.252531 0.011716 -1.320118 0.000000 0.000000 0.000000 1.000000"],
		["-0.307664 -6.150860 0.285363 -19.675442 0.000000 4.284058 0.410403 -22.728199 -7.493687 0.252533 -0.011716 0.320118 0.000000 0.000000 0.000000 1.000000"],
		["-0.160660 -2.892458 0.461196 -31.983715 0.000001 6.919527 0.192875 -7.501244 -7.498279 0.061975 -0.009882 0.191711 0.000000 0.000000 0.000000 1.000000"],
		["0.160658 -2.892459 -0.461196 32.583714 -0.000000 6.919526 -0.192875 19.501244 7.498279 0.061974 0.009882 -1.191711 0.000000 0.000000 0.000000 1.000000"],
		["-0.505623 3.712984 -0.433118 30.618282 -0.000000 6.511590 0.248097 -11.366772 7.482937 0.250887 -0.029266 1.548610 0.000000 0.000000 0.000000 1.000000"],
		["0.505624 3.712986 0.433118 -30.018284 -0.000001 6.511590 -0.248097 23.366772 -7.482937 0.250886 0.029266 -2.548610 0.000000 0.000000 0.000000 1.000000"],
		["-1.925925 6.375299 0.229941 -15.795859 0.000001 3.568785 -0.439766 36.783653 -7.248504 -1.693914 -0.061095 3.776662 0.000000 0.000000 0.000000 1.000000"],
		["1.925925 6.375299 -0.229941 16.395859 -0.000000 3.568784 0.439766 -24.783653 7.248505 -1.693914 0.061095 -4.776662 0.000000 0.000000 0.000000 1.000000"],
		["-2.942106 -6.619630 0.129521 -8.766449 0.000000 2.112106 0.479764 -27.583469 -6.898841 2.823032 -0.055236 3.366510 0.000000 0.000000 0.000000 1.000000"],
		["2.942104 -6.619631 -0.129521 9.366449 -0.000001 2.112104 -0.479764 39.583469 6.898842 2.823031 0.055236 -4.366510 0.000000 0.000000 0.000000 1.000000"],
		["-1.549021 -2.774371 -0.452909 32.003601 0.000000 6.943335 -0.189034 19.232370 7.338292 -0.585635 -0.095603 6.192231 0.000000 0.000000 0.000000 1.000000"],
		["1.549020 -2.774371 0.452909 -31.403601 0.000001 6.943335 0.189034 -7.232370 -7.338293 -0.585633 0.095603 -7.192231 0.000000 0.000000 0.000000 1.000000"],
		["-1.738145 1.189327 0.479881 -33.291691 -0.000000 7.399677 -0.081508 11.705523 -7.295811 -0.283344 -0.114326 7.502840 0.000000 0.000000 0.000000 1.000000"],
		["1.738144 1.189326 -0.479881 33.891689 -0.000000 7.399677 0.081507 0.294478 7.295811 -0.283343 0.114326 -8.502840 0.000000 0.000000 0.000000 1.000000"],
		["-3.400968 5.741309 -0.228238 16.276623 -0.000001 3.841195 0.429445 -24.061165 6.684566 2.921060 -0.116122 7.628573 0.000000 0.000000 0.000000 1.000000"],
		["3.400966 5.741311 0.228238 -15.676623 -0.000001 3.841195 -0.429445 36.061165 -6.684566 2.921057 0.116122 -8.628573 0.000000 0.000000 0.000000 1.000000"],
		["-6.635746 -3.161510 -0.099370 7.255907 -0.000001 3.198366 -0.452256 37.657909 3.495266 -6.002109 -0.188654 12.705755 0.000000 0.000000 0.000000 1.000000"],
		["6.635745 -3.161510 0.099370 -6.655906 0.000000 3.198365 0.452256 -25.657909 -3.495267 -6.002109 0.188654 -13.705755 0.000000 0.000000 0.000000 1.000000"],
		["-3.178397 -4.371010 0.346680 -23.967592 0.000001 5.741240 0.321719 -16.520317 -6.793217 2.045101 -0.162204 10.854267 0.000000 0.000000 0.000000 1.000000"],
		["3.178394 -4.371013 -0.346680 24.567591 -0.000000 5.741238 -0.321719 28.520317 6.793218 2.045099 0.162204 -11.854267 0.000000 0.000000 0.000000 1.000000"],
		["-2.916623 1.124330 -0.454504 32.115303 0.000001 7.400044 0.081359 0.304843 6.909653 0.474588 -0.191850 12.929502 0.000000 0.000000 0.000000 1.000000"],
		["2.916621 1.124330 0.454504 -31.515305 -0.000000 7.400044 -0.081359 11.695156 -6.909654 0.474589 0.191850 -13.929502 0.000000 0.000000 0.000000 1.000000"],
		["-4.294805 3.661346 0.329304 -22.751261 0.000002 6.025269 -0.297741 26.841841 -6.148549 -2.557474 -0.230021 15.601463 0.000000 0.000000 0.000000 1.000000"],
		["4.294803 3.661345 -0.329304 23.351259 -0.000000 6.025270 0.297741 -14.841841 6.148550 -2.557474 0.230021 -16.601463 0.000000 0.000000 0.000000 1.000000"],
		["-7.499684 0.063355 -0.001792 0.425472 0.000000 2.929983 0.460267 -26.218662 0.068824 6.903708 -0.195324 13.172678 0.000000 0.000000 0.000000 1.000000"],
		["7.499684 0.063354 0.001792 0.174528 -0.000000 2.929983 -0.460267 38.218662 -0.068823 6.903708 0.195324 -14.172678 0.000000 0.000000 0.000000 1.000000"],
		["-4.963970 -3.189996 -0.308638 21.904667 -0.000001 6.175848 -0.283697 25.858795 5.622188 -2.816527 -0.272504 18.575289 0.000000 0.000000 0.000000 1.000000"],
		["4.963968 -3.189997 0.308638 -21.304668 -0.000000 6.175848 0.283697 -13.858795 -5.622190 -2.816526 0.272504 -19.575289 0.000000 0.000000 0.000000 1.000000"],
		["-4.144962 -1.022679 0.411088 -28.476128 -0.000000 7.398932 0.081807 0.273499 -6.250543 0.678175 -0.272607 18.582497 0.000000 0.000000 0.000000 1.000000"],
		["4.144962 -1.022679 -0.411088 29.076126 -0.000000 7.398932 -0.081807 11.726501 6.250544 0.678176 0.272607 -19.582497 0.000000 0.000000 0.000000 1.000000"],
		["-5.083177 3.096819 -0.304200 21.594027 -0.000001 6.205755 0.280781 -13.654707 5.514645 2.854524 -0.280400 19.127966 0.000000 0.000000 0.000000 1.000000"],
		["5.083176 3.096823 0.304200 -20.994028 0.000001 6.205754 -0.280782 25.654707 -5.514647 2.854525 0.280399 -20.127966 0.000000 0.000000 0.000000 1.000000"],
		["-7.326222 1.127744 0.076148 -5.030359 -0.000001 5.337012 -0.351292 30.590412 -1.605140 -5.147280 -0.347557 23.828972 0.000000 0.000000 0.000000 1.000000"],
		["7.326222 1.127744 -0.076148 5.630359 -0.000000 5.337012 0.351292 -18.590412 1.605139 -5.147281 0.347557 -24.828972 0.000000 0.000000 0.000000 1.000000"],
		["-6.474771 -2.616503 0.182346 -12.464223 -0.000001 5.419583 0.345627 -18.193922 -3.785149 4.475716 -0.311916 21.334120 0.000000 0.000000 0.000000 1.000000"],
		["6.474770 -2.616504 -0.182346 13.064223 -0.000000 5.419581 -0.345628 30.193922 3.785150 4.475718 0.311916 -22.334120 0.000000 0.000000 0.000000 1.000000"],
		["-5.451219 -0.508029 -0.341735 24.221445 -0.000000 7.463435 -0.049312 9.451858 5.151137 -0.537624 -0.361643 24.814997 0.000000 0.000000 0.000000 1.000000"],
		["5.451219 -0.508028 0.341735 -23.621447 -0.000000 7.463435 0.049312 2.548143 -5.151137 -0.537624 0.361643 -25.814997 0.000000 0.000000 0.000000 1.000000"],
		["-5.935410 0.985898 0.298507 -20.595474 -0.000000 7.324550 -0.107517 13.526174 -4.584857 -1.276312 -0.386437 26.550617 0.000000 0.000000 0.000000 1.000000"],
		["5.935410 0.985898 -0.298507 21.195473 -0.000000 7.324550 0.107517 -1.526174 4.584857 -1.276312 0.386437 -27.550617 0.000000 0.000000 0.000000 1.000000"],
		["-7.250820 1.221711 -0.098500 7.195035 -0.000000 5.779980 0.318621 -16.303457 1.917187 4.620525 -0.372530 25.577082 0.000000 0.000000 0.000000 1.000000"],
		["7.250820 1.221712 0.098501 -6.595035 0.000000 5.779980 -0.318621 28.303457 -1.917188 4.620525 0.372530 -26.577082 0.000000 0.000000 0.000000 1.000000"],
		["-7.281588 -0.772892 -0.108138 7.869672 -0.000001 6.770683 -0.215075 21.055218 1.796797 -3.132169 -0.438234 30.176382 0.000000 0.000000 0.000000 1.000000"],
		["7.281588 -0.772891 0.108138 -7.269671 -0.000000 6.770683 0.215075 -9.055218 -1.796797 -3.132170 0.438234 -31.176382 0.000000 0.000000 0.000000 1.000000"],
		["-6.724941 -0.962456 0.211858 -14.530052 0.000001 7.178018 0.144930 -4.145110 -3.320417 1.949293 -0.429082 29.535753 0.000000 0.000000 0.000000 1.000000"],
		["6.724941 -0.962458 -0.211858 15.130053 -0.000000 7.178017 -0.144930 16.145111 3.320418 1.949295 0.429082 -30.535753 0.000000 0.000000 0.000000 1.000000"],
		["-6.935052 0.578879 -0.186428 13.349942 0.000001 7.344292 0.101355 -1.094834 2.855705 1.405801 -0.452738 31.191675 0.000000 0.000000 0.000000 1.000000"],
		["6.935052 0.578879 0.186428 -12.749942 -0.000000 7.344292 -0.101355 13.094834 -2.855704 1.405802 0.452738 -32.191673 0.000000 0.000000 0.000000 1.000000"],
		["-7.418036 0.175292 0.072786 -4.795047 0.000001 7.405165 -0.079261 11.548292 -1.105778 -1.175927 -0.488282 33.679771 0.000000 0.000000 0.000000 1.000000"],
		["7.418036 0.175291 -0.072786 5.395048 -0.000000 7.405166 0.079261 0.451708 1.105778 -1.175925 0.488283 -34.679771 0.000000 0.000000 0.000000 1.000000"],
		["-7.500000 0.000001 0.000000 0.300000 0.000001 7.471460 0.043578 2.949549 -0.000001 0.653668 -0.498097 34.366814 0.000000 0.000000 0.000000 1.000000"],
		["7.500000 0.000000 0.000000 0.300000 0.000000 7.471460 -0.043578 9.050451 0.000000 0.653668 0.498097 -35.366814 0.000000 0.000000 0.000000 1.000000"],
	]
	raw_string = light_positions[camera_idx][0]
	float_values = list(map(float, raw_string.strip().split()))
	matrix_4x4 = np.array(float_values).reshape((4, 4))
	params['arealight.to_world'] = mi.Transform4f(matrix_4x4)
	params.update()
	# print("after(no T):\n", np.array(params['arealight.to_world']))

def save_images(scene, params, dir, spp):
    print("start rendering")
    for i, sensor in enumerate(scene.sensors()):
					_update_light_position(params, i)
											
					integrator = mi.load_dict({
							'type': 'prb_path_volume',
							'max_sss_depth': 256,
							'max_path_depth': 10,
					})
					image = mi.render(scene, sensor=sensor, integrator=integrator, spp=spp)
					bitmap = mi.Bitmap(image).convert(
									pixel_format=mi.Bitmap.PixelFormat.RGB,
									component_format=mi.Struct.Type.UInt8,
									srgb_gamma=True
					)
					png_path = dir / f"ref_view_{i:03d}.png"
					bitmap.write(str(png_path))

					bitmap = mi.Bitmap(image).convert(
													pixel_format=mi.Bitmap.PixelFormat.RGB,
													component_format=mi.Struct.Type.Float32,
													srgb_gamma=False
					)
					exr_path = dir / f"ref_view_{i:03d}.exr"
					bitmap.write(str(exr_path))
					print(f"saving: {i}")

In [4]:
ref_spp=4096
scene_name = 'kiwi'
ref_dir=Path("third_party/kiwi/references")
ref_scene_path=f'third_party/{scene_name}/mts_scene/kiwi_ref.xml'
ref_scene = mi.load_file(ref_scene_path)
ref_params = mi.traverse(ref_scene)

save_images(ref_scene, ref_params, ref_dir, ref_spp)

start rendering
saving: 0
saving: 1
saving: 2
saving: 3
saving: 4
saving: 5
saving: 6
saving: 7
saving: 8
saving: 9
saving: 10
saving: 11
saving: 12
saving: 13
saving: 14
saving: 15
saving: 16
saving: 17
saving: 18
saving: 19
saving: 20
saving: 21
saving: 22
saving: 23
saving: 24
saving: 25
saving: 26
saving: 27
saving: 28
saving: 29
saving: 30
saving: 31
saving: 32
saving: 33
saving: 34
saving: 35
saving: 36
saving: 37
saving: 38
saving: 39
saving: 40
saving: 41
saving: 42
saving: 43
saving: 44
saving: 45
saving: 46
saving: 47
saving: 48
saving: 49
saving: 50
saving: 51
saving: 52
saving: 53
saving: 54
saving: 55
saving: 56
saving: 57
saving: 58
saving: 59
saving: 60
saving: 61
saving: 62
saving: 63
saving: 64
saving: 65
saving: 66
saving: 67
saving: 68
saving: 69
saving: 70
saving: 71
saving: 72
saving: 73
saving: 74
saving: 75
saving: 76
saving: 77
saving: 78
saving: 79
saving: 80
saving: 81
saving: 82
saving: 83
saving: 84
saving: 85
saving: 86
saving: 87
saving: 88
saving: 89
savi

In [ ]:
def format_float(f):
  """Formats a float such that 0.1 becomes "0_1", 10.0 becomes "10_0", etc."""
  return str(f).replace('.', '_')

scene_name = 'kiwi'
technique = 'mipmap_pyramid'
base_learning_rate = 0.0005

skip_existing = False

print(
    f'******** Running {technique} with base learning rate'
    f' {base_learning_rate} ********'
)

override_bindings = []
result_folder = f'results/{scene_name}/{technique}'

result_folder += f'_lr_{format_float(base_learning_rate)}'
override_bindings.append(
    f'SceneConfig.base_learning_rate={base_learning_rate}'
)

# Ensure that the default tmp folder is used
override_bindings.append("SceneConfig.tmp_folder=''")
override_bindings.append(
    f'SceneConfig.use_gradient_filtering=False'
)

override_bindings.append(
    f"SceneConfig.result_folder='{result_folder}'"
)

if technique == 'gradient_filtering':
    gin_config_name = f'{scene_name}/naive'
else:
    gin_config_name = f'{scene_name}/{technique}'

print(f'Next result location: {result_folder}')
if skip_existing and Path(result_folder).exists():
    print('Skipping, already present')
else:
    # Run the config
    optimization_cli.run_config(gin_config_name, override_bindings, sss_config=True)

******** Running mipmap_pyramid with base learning rate 0.0005 ********
Next result location: results/kiwi/mipmap_pyramid_lr_0_0005
Preparing Mitsuba scene for optimization
Skipping already existing mitsuba scene: tmp\kiwi\mts_scene
Preparing references and sensors for optimization
Rendering camera elm__1
Reference found locally: tmp\kiwi\references\ref_view_000.exr
Rendering camera elm__2
Reference found locally: tmp\kiwi\references\ref_view_001.exr
Rendering camera elm__3
Reference found locally: tmp\kiwi\references\ref_view_002.exr
Rendering camera elm__4
Reference found locally: tmp\kiwi\references\ref_view_003.exr
Rendering camera elm__5
Reference found locally: tmp\kiwi\references\ref_view_004.exr
Rendering camera elm__6
Reference found locally: tmp\kiwi\references\ref_view_005.exr
Rendering camera elm__7
Reference found locally: tmp\kiwi\references\ref_view_006.exr
Rendering camera elm__8
Reference found locally: tmp\kiwi\references\ref_view_007.exr
Rendering camera elm__9
Refer

Loading references for sensor elm__100: 100%|██████████| 1/1 [00:00<00:00, 986.20it/s]


Preparing optimization variables
Optimizing mat-kiwi's single_scattering_albedo from default value : [[0.5, 0.5, 0.5]]
Optimizing mat-kiwi's extinction_coefficient from default value : [[10, 10, 10]]
Optimizing mat-kiwi's hg_coefficient from default value : [[0, 0, 0]]
Learning rate for (texture) mat-kiwi (single_scattering_albedo) is  0.0005
Learning rate for (scalar) mat-kiwi (extinction_coefficient) is  0.5
Learning rate for (texture) mat-kiwi (hg_coefficient) is  0.0005
Starting optimization
Resizing initial value for mat-kiwi.nested_bsdf.single_scattering_albedo.data/mat-kiwi.nested_bsdf.single_scattering_albedo.data to shape: (2048, 2048, 3)
Skipping resizing as target shape is already reached.
Resizing initial value for mat-kiwi.nested_bsdf.hg_coefficient.data/mat-kiwi.nested_bsdf.hg_coefficient.data to shape: (512, 512, 3)
Skipping resizing as target shape is already reached.


Rendering at resolution level 0 (width=436):   7%|▋         | 300/4096 [09:06<1:17:25,  1.22s/it] 

# Figure starts here

In [ ]:
import matplotlib.pyplot as plt
from practical_reconstruction import figutils
from matplotlib import gridspec

def figure_grid_setup(image_shape,image_crop_shape,n_columns = 7,inner_space=0.0,outer_space=0.1,figwidth=figutils.TEXT_WIDTH):
  # Image aspect ratios
  h, w = image_shape
  h_crop, w_crop = image_crop_shape

  top_inner_rows = 1
  top_inner_cols = n_columns
  # Spacing in the inner gridspec
  top_inner_wspace = inner_space
  # same vertical spacing as horizontal spacing
  top_inner_hspace = top_inner_wspace * figutils.gridspec_aspect(
      n_rows=1, n_cols=1, w=[w]*top_inner_cols, h=[h]
  )
  top_height_ratios = [h]
  top_inner_aspect = figutils.gridspec_aspect(
      n_rows=top_inner_rows,
      n_cols=top_inner_cols,
      w=[w] * top_inner_cols,
      h=top_height_ratios,
      wspace=top_inner_wspace,
      hspace=top_inner_hspace,
  )
  # print(top_inner_aspect)

  # Spacing in the main griddpec
  outer_rows = 3
  outer_cols = 1
  outer_wspace = 0.0
  outer_hspace = outer_space
  # If width is 1, we need the sum of the inverses for the height (single column)
  # If height is 1, we need the sum for the width (single row)
  outer_aspect = figutils.gridspec_aspect(
      n_rows=outer_rows,
      n_cols=outer_cols,
      w=1,
      h=[1 / top_inner_aspect] * outer_rows,
      wspace=outer_wspace,
      hspace=outer_hspace,
  )

  fig = plt.figure(
      1, figsize=(figwidth, figwidth / outer_aspect)
  )

  outer_gs = fig.add_gridspec(
      outer_rows,
      outer_cols,
      hspace=outer_hspace,
      wspace=outer_wspace,
      height_ratios=[1 / top_inner_aspect] * outer_rows,
      width_ratios=[1]*outer_cols,
  )

  top_inner_gss = []
  for row in range(outer_rows):
    top_inner_gs = gridspec.GridSpecFromSubplotSpec(
        top_inner_rows,
        top_inner_cols,
        subplot_spec=outer_gs[row],
        wspace=top_inner_wspace,
        hspace=top_inner_wspace,
        width_ratios=[h] * top_inner_cols,
        height_ratios=top_height_ratios,
    )
    top_inner_gss.append(top_inner_gs)

  return (
      fig,
      (top_inner_gss, top_inner_rows, top_inner_cols),
  )

In [ ]:
import drjit as dr
import numpy as np
from core import mitsuba_io
from core import image_util

def l2_error(ref, img):
  return dr.mean(dr.square(ref - img)).array[0]


def l1_error(ref, img):
  return dr.mean(dr.abs(ref - img)).array[0]

sensor_indices = [20, 1, 25, 12]

lr = '0_0005'
normalmap_lr = '0_01'
scene_folder = 'results/kiwi'

ref_scene_folder = (
    'third_party/kiwi/unmasked_references'
)
ref_masked_scene_folder = (
    'third_party/kiwi/references'
)
deng_scene_folder = (
    'third_party/kiwi/deng_optimized/'
)

crop_sizes = [
    (110, 110),
    (126, 126),
    (94, 94),
    (113, 113),
]

crop_offsets = [
    (159, 92),
    (150, 75),
    (168, 96),
    (157, 86),
]

our_images = []
ref_masked_images = []
ref_images = []
deng_images = []
errors_l2_ours = []
errors_l2_deng = []

boost = np.sqrt(2)

for sensor_idx, crop_size, crop_offset in zip(sensor_indices,crop_sizes,crop_offsets):

  our_filename = f'{scene_folder}/mipmap_pyramid_lr_{lr}/frames/camera_{sensor_idx:03d}_iter_4095_spp_8192.exr'
  our_image = mitsuba_io.read_bitmap(our_filename).convert(
      pixel_format=mi.Bitmap.PixelFormat.RGB,
      component_format=mi.Struct.Type.Float32,
  )
  our_images.append(image_util.tonemap(boost*our_image))
  ref_masked_filename = (
      f'{ref_masked_scene_folder}/ref_view_{sensor_idx:03d}.exr'
  )
  ref_masked_image = mitsuba_io.read_bitmap(ref_masked_filename).convert(
      pixel_format=mi.Bitmap.PixelFormat.RGB,
      component_format=mi.Struct.Type.Float32,
  )
  ref_masked_images.append(image_util.tonemap(ref_masked_image))

  ref_filename = f'{ref_scene_folder}/kiwi_{sensor_idx:05d}.exr'
  ref_image = mitsuba_io.read_bitmap(ref_filename).convert(
      pixel_format=mi.Bitmap.PixelFormat.RGB,
      component_format=mi.Struct.Type.Float32,
  )
  ref_images.append(
      image_util.tonemap(
          boost*image_util.resize_to_width(ref_image, ref_image.size()[0] // 2)
      )
  )
  deng_filename = f'{deng_scene_folder}/optimized_{sensor_idx:03d}_spp_256.exr'
  deng_image = mitsuba_io.read_bitmap(deng_filename).convert(
      pixel_format=mi.Bitmap.PixelFormat.RGB,
      component_format=mi.Struct.Type.Float32,
  )
  deng_images.append(image_util.tonemap(boost*deng_image))

  errors_l2_ours.append(
      l2_error(
          figutils.crop_image(
              mi.TensorXf(ref_masked_image), crop_offset, crop_size
          ),
          figutils.crop_image(
              mi.TensorXf(our_image), crop_offset, crop_size
          ),
      )
  )
  errors_l2_deng.append(
      l2_error(
          figutils.crop_image(
              mi.TensorXf(ref_masked_image), crop_offset, crop_size
          ),
          figutils.crop_image(
              mi.TensorXf(deng_image), crop_offset, crop_size
          ),
      )
  )

our_images = [
    figutils.crop_image(np.array(image), crop_offsets[i], crop_sizes[i])
    for i,image in enumerate(our_images)
]
ref_images = [
    figutils.crop_image(np.array(image), crop_offsets[i], crop_sizes[i])
    for i,image in enumerate(ref_images)
]
deng_images = [
    figutils.crop_image(np.array(image), crop_offsets[i], crop_sizes[i])
    for i,image in enumerate(deng_images)
]

FileNotFoundError: [Errno 2] No such file or directory: 'results/kiwi/mipmap_pyramid_lr_0_0005/frames/camera_020_iter_4095_spp_8192.exr'

In [ ]:
FIGURE_DIR = "figures/pdfs"
FIGURE_NAME = "deng_comparison"

def error_format(error, scale):
  return f"{error*scale:.3f}"

(
    fig,
    (top_inner_gss, top_inner_rows, top_inner_cols),
) = figure_grid_setup(
    our_images[0].shape[:2],
    our_images[0].shape[:2],
    n_columns=len(sensor_indices),
    inner_space=0.03,
    outer_space=0.15,
    figwidth=figutils.COLUMN_WIDTH,
)


scale = 1000
scale_txt = figutils.math_label(r"\text{$\times 10^3$}")

line_width = 0.75
crop_color = 'orange'
crop_color = (55/255.0,118/255.0,171/255.0,1.0)

row_titles = [
    "Deng et al. 2022",
    "Ours",
    "Reference",
]

for row, top_inner_gs in enumerate(top_inner_gss):
  if row == 0:
    images = deng_images
    errors = errors_l2_deng
  elif row == 1:
    images = our_images
    errors = errors_l2_ours
  elif row == 2:
    images = ref_images

  for col in range(top_inner_cols):
    ax = fig.add_subplot(top_inner_gs[col])
    ax.spines[:].set_color(crop_color)
    ax.spines[:].set_linewidth(line_width)
    figutils.disable_ticks(ax)
    image = images[col]

    # alpha_channel = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8) + 255
    # black_pixels = np.all(image == 0, axis=-1)
    # alpha_channel[black_pixels] = 0
    # image_with_alpha = np.dstack((image, alpha_channel))

    ax.imshow(image, aspect="equal")
    if col == 0:
      ax.set_ylabel(row_titles[row],labelpad=1.5)

    if row == 0 or row == 1:
      label = error_format(errors[col], scale)
    elif row == 2:
      label = r"$L_2$ " + scale_txt + " error" if col == 0 else ""
    ax.set_xlabel(label, labelpad=1.5)

figutils.force_post_crop_size(fig, figutils.COLUMN_WIDTH)

In [ ]:
figutils.savefig(
    fig,
    name=Path(FIGURE_NAME),
    fig_directory=Path(FIGURE_DIR),
    dpi=300,
    pad_inches=0.005,
    bbox_inches="tight",
    compress=False,
    target_width=figutils.COLUMN_WIDTH,
    backend=None,
)